FROM:
https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

# Keras Tutorial Overview
There is not a lot of code required, but we are going to step over it slowly so that you will know how to create your own models in the future.

### The steps you are going to cover in this tutorial are as follows:

Load Data.
Define Keras Model.
Compile Keras Model.
Fit Keras Model.
Evaluate Keras Model.
Tie It All Together.
Make Predictions

### This Keras tutorial has a few requirements:

You have Python 2 or 3 installed and configured.
You have SciPy (including NumPy) installed and configured.
You have Keras and a backend (Theano or TensorFlow) installed and configured.
If you need help with your environment, see the tutorial:

How to Setup a Python Environment for Deep Learning
Create a new file called keras_first_network.py and type or copy-and-paste the code into the file as you go.

# 1. Load Data
The first step is to define the functions and classes we intend to use in this tutorial.

We will use the NumPy library to load our dataset and we will use two classes from the Keras library to define our model.

The imports required are listed below.

In [1]:

# first neural network with keras tutorial
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense


We can now load our dataset.

In this Keras tutorial, we are going to use the Pima Indians onset of diabetes dataset. This is a standard machine learning dataset from the UCI Machine Learning repository. It describes patient medical record data for Pima Indians and whether they had an onset of diabetes within five years.

As such, it is a binary classification problem (onset of diabetes as 1 or not as 0). All of the input variables that describe each patient are numerical. This makes it easy to use directly with neural networks that expect numerical input and output values, and ideal for our first neural network in Keras.

The dataset is available from here:

Dataset CSV File (pima-indians-diabetes.csv)
Dataset Details

Download the dataset and place it in your local working directory, the same location as your python file.

Save it with the filename: 'pima-indians-diabetes.csv'



We can now load the file as a matrix of numbers using the NumPy function loadtxt().

There are eight input variables and one output variable (the last column). We will be learning a model to map rows of input variables (X) to an output variable (y), which we often summarize as y = f(X).

The variables can be summarized as follows:

Input Variables (X):

1. Number of times pregnant
2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
3. Diastolic blood pressure (mm Hg)
4. Triceps skin fold thickness (mm)
5. 2-Hour serum insulin (mu U/ml)
6. Body mass index (weight in kg/(height in m)^2)
7. Diabetes pedigree function
8. Age (years)

Output Variables (y):

1. Class variable (0 or 1)

Once the CSV file is loaded into memory, we can split the columns of data into input and output variables.


In [2]:
dataset = loadtxt('pima-indians-diabetes.csv', delimiter = ',')
# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]


We are now ready to define our neural network model.

# 2. Define Keras Model
Models in Keras are defined as a sequence of layers.

We create a Sequential model and add layers one at a time until we are happy with our network architecture.

The first thing to get right is to ensure the input layer has the right number of input features. This can be specified when creating the first layer with the input_dim argument and setting it to 8 for the 8 input variables.

How do we know the number of layers and their types?

This is a very hard question. There are heuristics that we can use and often the best network structure is found through a process of trial and error experimentation (I explain more about this here). Generally, you need a network large enough to capture the structure of the problem.

In this example, we will use a fully-connected network structure with three layers.

Fully connected layers are defined using the Dense class. We can specify the number of neurons or nodes in the layer as the first argument, and specify the activation function using the activation argument.

We will use the rectified linear unit activation function referred to as ReLU on the first two layers and the Sigmoid function in the output layer.

It used to be the case that Sigmoid and Tanh activation functions were preferred for all layers. These days, better performance is achieved using the ReLU activation function. We use a sigmoid on the output layer to ensure our network output is between 0 and 1 and easy to map to either a probability of class 1 or snap to a hard classification of either class with a default threshold of 0.5.

We can piece it all together by adding each layer:

- The model expects rows of data with 8 variables (the input_dim=8 argument)
- The first hidden layer has 12 nodes and uses the relu activation function.
- The second hidden layer has 8 nodes and uses the relu activation function.
- The output layer has one node and uses the sigmoid activation function.


In [3]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

### Note, the most confusing thing here is that the shape of the input to the model is defined as an argument on the first hidden layer. 
This means that the line of code that adds the first Dense layer is doing 2 things, defining the input or visible layer and the first hidden layer.

# 3. Compile Keras Model
Now that the model is defined, we can compile it.

Compiling the model uses the efficient numerical libraries under the covers (the so-called backend) such as Theano or TensorFlow. The backend automatically chooses the best way to represent the network for training and making predictions to run on your hardware, such as CPU or GPU or even distributed.

When compiling, we must specify some additional properties required when training the network. Remember training a network means finding the best set of weights to map inputs to outputs in our dataset.

We must specify the loss function to use to evaluate a set of weights, the optimizer is used to search through different weights for the network and any optional metrics we would like to collect and report during training.

In this case, we will use cross entropy as the loss argument. This loss is for a binary classification problems and is defined in Keras as “binary_crossentropy“. You can learn more about choosing loss functions based on your problem here:

- How to Choose Loss Functions When Training Deep Learning Neural Networks

We will define the optimizer as the efficient stochastic gradient descent algorithm “adam“. This is a popular version of gradient descent because it automatically tunes itself and gives good results in a wide range of problems. To learn more about the Adam version of stochastic gradient descent see the post:

- Gentle Introduction to the Adam Optimization Algorithm for Deep Learning

Finally, because it is a classification problem, we will collect and report the classification accuracy, defined via the metrics argument.

In [4]:

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. Fit Keras Model
We have defined our model and compiled it ready for efficient computation.

Now it is time to execute the model on some data.

We can train or fit our model on our loaded data by calling the fit() function on the model.

Training occurs over epochs and each epoch is split into batches.

-Epoch: One pass through all of the rows in the training dataset.
-Batch: One or more samples considered by the model within an epoch before weights are updated.

One epoch is comprised of one or more batches, based on the chosen batch size and the model is fit for many epochs. For more on the difference between epochs and batches, see the post:

-What is the Difference Between a Batch and an Epoch in a Neural Network?

The training process will run for a fixed number of iterations through the dataset called epochs, that we must specify using the epochs argument. We must also set the number of dataset rows that are considered before the model weights are updated within each epoch, called the batch size and set using the batch_size argument.

For this problem, we will run for a small number of epochs (150) and use a relatively small batch size of 10.

These configurations can be chosen experimentally by trial and error. We want to train the model enough so that it learns a good (or good enough) mapping of rows of input data to the output classification. The model will always have some error, but the amount of error will level out after some point for a given model configuration. This is called model convergence.

In [5]:

# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10)

Epoch 1/150
77/77 [==============================] - 0s 393us/step - loss: 9.0450 - accuracy: 0.5781
Epoch 2/150
77/77 [==============================] - 0s 389us/step - loss: 2.1278 - accuracy: 0.5247
Epoch 3/150
77/77 [==============================] - 0s 402us/step - loss: 1.0655 - accuracy: 0.6120
Epoch 4/150
77/77 [==============================] - 0s 389us/step - loss: 0.8929 - accuracy: 0.6510
Epoch 5/150
77/77 [==============================] - 0s 402us/step - loss: 0.8788 - accuracy: 0.6497
Epoch 6/150
77/77 [==============================] - 0s 376us/step - loss: 0.8472 - accuracy: 0.6354
Epoch 7/150
77/77 [==============================] - 0s 389us/step - loss: 0.7789 - accuracy: 0.6484
Epoch 8/150
77/77 [==============================] - 0s 402us/step - loss: 0.7988 - accuracy: 0.6250
Epoch 9/150
77/77 [==============================] - 0s 389us/step - loss: 0.7634 - accuracy: 0.6341
Epoch 10/150
77/77 [==============================] - 0s 389us/step - loss: 0.7040 - accura

77/77 [==============================] - 0s 389us/step - loss: 0.5330 - accuracy: 0.7174
Epoch 82/150
77/77 [==============================] - 0s 389us/step - loss: 0.5207 - accuracy: 0.7461
Epoch 83/150
77/77 [==============================] - 0s 363us/step - loss: 0.5094 - accuracy: 0.7487
Epoch 84/150
77/77 [==============================] - 0s 337us/step - loss: 0.5232 - accuracy: 0.7357
Epoch 85/150
77/77 [==============================] - 0s 389us/step - loss: 0.5020 - accuracy: 0.7461
Epoch 86/150
77/77 [==============================] - 0s 389us/step - loss: 0.5522 - accuracy: 0.7240
Epoch 87/150
77/77 [==============================] - 0s 397us/step - loss: 0.5307 - accuracy: 0.7214
Epoch 88/150
77/77 [==============================] - 0s 376us/step - loss: 0.5072 - accuracy: 0.7448
Epoch 89/150
77/77 [==============================] - 0s 389us/step - loss: 0.5210 - accuracy: 0.7357
Epoch 90/150
77/77 [==============================] - 0s 389us/step - loss: 0.5123 - accuracy: 

This is where the work happens on your CPU or GPU.

No GPU is required for this example, but if you’re interested in how to run large models on GPU hardware cheaply in the cloud, see this post:

# 5. Evaluate Keras Model
We have trained our neural network on the entire dataset and we can evaluate the performance of the network on the same dataset.

This will only give us an idea of how well we have modeled the dataset (e.g. train accuracy), but no idea of how well the algorithm might perform on new data. We have done this for simplicity, but ideally, you could separate your data into train and test datasets for training and evaluation of your model.

You can evaluate your model on your training dataset using the evaluate() function on your model and pass it the same input and output used to train the model.

This will generate a prediction for each input and output pair and collect scores, including the average loss and any metrics you have configured, such as accuracy.

The evaluate() function will return a list with two values. The first will be the loss of the model on the dataset and the second will be the accuracy of the model on the dataset. We are only interested in reporting the accuracy, so we will ignore the loss value.

In [6]:

# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

24/24 [==============================] - 0s 374us/step - loss: 0.4561 - accuracy: 0.7695
Accuracy: 76.95


# 6. Tie It All Together
You have just seen how you can easily create your first neural network model in Keras.

Let’s tie it all together into a complete code example.

In [7]:
# first neural network with keras tutorial
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
# load the dataset
dataset = loadtxt('pima-indians-diabetes.csv', delimiter=',')
# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]
# define the keras model
model = Sequential()
model.add(Dense(24, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10, verbose=0)
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

24/24 [==============================] - 0s 457us/step - loss: 0.4585 - accuracy: 0.7760
Accuracy: 77.60


# 7. Make Predictions
The number one question I get asked is:

After I train my model, how can I use it to make predictions on new data?

Great question.

We can adapt the above example and use it to generate predictions on the training dataset, pretending it is a new dataset we have not seen before.

Making predictions is as easy as calling the predict() function on the model. We are using a sigmoid activation function on the output layer, so the predictions will be a probability in the range between 0 and 1. We can easily convert them into a crisp binary prediction for this classification task by rounding them.

For example:

In [8]:
# make probability predictions with the model
predictions = model.predict(X)
# round predictions 
rounded = [round(x[0]) for x in predictions]

Alternately, we can call the predict_classes() function on the model to predict crisp classes directly, for example:

In [9]:
# make class predictions with the model
predictions = model.predict_classes(X)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


The complete example below makes predictions for each example in the dataset, then prints the input data, predicted class and expected class for the first 5 examples in the dataset.

In [10]:
# first neural network with keras make predictions
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
# load the dataset
dataset = loadtxt('pima-indians-diabetes.csv', delimiter=',')
# split into input (X) and output (y) variables
split = int(dataset.shape[0] * 0.8)
X = dataset[0:split,0:8]
X_test = dataset[split:,0:8]
y = dataset[:split,8]
y_test = dataset[split:,8]
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10, verbose=0)
# make class predictions with the model
y_pred = model.predict_classes(X_test)



In [12]:
_, accuracy1 = model.evaluate(X,y)
_, accuracy2 = model.evaluate(X_test, y_test)

print(accuracy1, accuracy2)

    

5/5 [==============================] - 0s 600us/step - loss: 0.5834 - accuracy: 0.6948
0.7638436555862427 0.6948052048683167


In [16]:
# Visualize training history
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
history = model.fit(X, Y, validation_split=0.33, epochs=150, batch_size=10, verbose=0)
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

ModuleNotFoundError: No module named 'matplotlib'